In [48]:
import os
import azureml.core
import pandas as pd
from azureml.core.runconfig import JarLibrary
from azureml.core.compute import ComputeTarget, DatabricksCompute
from azureml.exceptions import ComputeTargetException
from azureml.core.datastore import Datastore
from azureml.data.data_reference import DataReference
from azureml.core.databricks import PyPiLibrary

from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.core import Workspace, Environment, Experiment, Datastore, Dataset, ScriptRunConfig
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
from azureml.pipeline.steps import DatabricksStep, PythonScriptStep
from azureml.train.hyperdrive import choice, loguniform

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)


SDK version: 1.44.0


In [49]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


scsccps-dsai-aide-dev-mlw
scsc-dsai-aide-dev-rg
canadacentral
105efa68-0ff4-486f-ae3a-86e28a447237


In [50]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)

from azure.identity import DefaultAzureCredential


In [51]:
ml_client = MLClient(
    DefaultAzureCredential(), ws.subscription_id, ws.resource_group, ws.name
)


In [52]:
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is the online endpoint for the service desk model",
    auth_mode="key"
)


In [53]:
ml_client.online_endpoints.begin_create_or_update(endpoint)


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-09131701520496.canadacentral.inference.ml.azure.com/score', 'swagger_uri': 'https://endpoint-09131701520496.canadacentral.inference.ml.azure.com/swagger.json', 'name': 'endpoint-09131701520496', 'description': 'this is the online endpoint for the service desk model', 'tags': {'ClientOrganization': 'Sc', 'CloudUsageProfile': '1', 'DataSensitivity': 'Unclassified', 'Environment': 'Sandbox', 'ProjectEmail': 'Scott.Syms@ssc-spc.gc.ca', 'ProjectName': 'DSAI', 'TechnicalEmail': 'Scott.Syms@ssc-spc.gc.ca'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/105efa68-0ff4-486f-ae3a-86e28a447237/resourcegroups/scsc-dsai-aide-dev-rg/providers/microsoft.machinelearningservices/workspaces/scsccps-dsai-aide-dev-mlw/onlineendpoints/endpoint-09131701520496', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/105efa68-0ff4-486f-ae3a-86e28a447237/prov

In [54]:
# env = ml_client.environments.get(name='Transformer-DeBerta', label='latest')

env = Environment(
    image="4b7c2058429e460f92f578b9aa2304e0.azurecr.io/azureml/azureml_4e49e7118305b1846866bfb3b7c7290f",
)


In [56]:
dir = 'output'
prefix_path = "outputs/model"
model_directory = f'{dir}/{prefix_path}'

model = Model(path=model_directory)


In [57]:
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code=model_directory, scoring_script="score.py"
    ),
    instance_type="Standard_F2s_v2",
    instance_count=1,
)


In [58]:
ml_client.begin_create_or_update(blue_deployment)


Check: endpoint endpoint-09131701520496 exists
Your file exceeds 100 MB. If you experience low upload speeds or latency, we recommend using the AzCopy tool for this file transfer. See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading model (433.93 MBs): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

..........................................................................................................................................................................................................

Done (18m 28s)


In [64]:
endpoint_dep = ml_client.online_deployments.get(endpoint_name='endpoint-09131701520496', name='blue')
endpoint_dep

ManagedOnlineDeployment({'private_network_connection': False, 'egress_public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'endpoint_name': 'endpoint-09131701520496', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {'ClientOrganization': 'Sc', 'CloudUsageProfile': '1', 'DataSensitivity': 'Unclassified', 'Environment': 'Sandbox', 'ProjectEmail': 'Scott.Syms@ssc-spc.gc.ca', 'ProjectName': 'DSAI', 'TechnicalEmail': 'Scott.Syms@ssc-spc.gc.ca'}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/105efa68-0ff4-486f-ae3a-86e28a447237/providers/Microsoft.MachineLearningServices/locations/canadacentral/mfeOperationsStatus/od:4b7c2058-429e-460f-92f5-78b9aa2304e0:28d78c82-f340-4584-9537-93fdde779304?api-version=2022-02-01-preview'}, 'id': '/subscriptions/105efa68-0ff4-486f-ae3a-86e28a447237/resourceGroups/scsc-dsai-aide-dev-rg/providers/Microsoft.MachineLearningServices/workspaces/scsccps-dsai-aide-dev-mlw/onlineEndpoints/endpoint

In [78]:
result = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file='../project/sample_request.json')

In [81]:
list(eval(result))

['EDC00075', 'ITS00340']

In [87]:
deploy_name = "blue"

li_to_be_deleted_dep = []
for deployment in ml_client.online_deployments.list(online_endpoint_name):
    if deploy_name != i.name:
        li_to_be_deleted_dep.append(i.name)

In [ ]:
for online_deployment in li_to_be_deleted_dep:
    ml_client.online_deployments.delete(name=online_deployment, endpoint_name=online_endpoint_name)


In [88]:
ws.write_config()

In [89]:
ml_client2 = MLClient.from_config()

TypeError: from_config() missing 1 required positional argument: 'credential'